In [1]:
import numpy as np
import tensorflow as tf
import keras
import cv2
from keras.layers import MaxPool2D,Conv2D,UpSampling2D,Input,Dropout
from keras.models import Sequential
from keras.utils import img_to_array, plot_model
from keras.utils.vis_utils import plot_model
import os
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import gc

In [2]:
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)
# defining the size of the image
SIZE = 256
color_img = []
path = 'E:/TER/landscape Images/color'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):    
    if i == '7129.jpg':
        break
    else:    
        img = cv2.imread(path + '/'+i,1)
        # open cv reads images in BGR format so we have to convert it to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #resizing image
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0
        color_img.append(img_to_array(img))


gray_img = []
path = 'E:/TER/landscape Images/gray'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
     if i == '7129.jpg':
        break
     else: 
        img = cv2.imread(path + '/'+i,1)

        #resizing image
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0
        gray_img.append(img_to_array(img))

100%|██████████| 7129/7129 [00:16<00:00, 431.69it/s]


In [3]:
test_gray_image = gray_img
test_color_image = color_img
del gray_img
del color_img
gc.collect()
test_gray_image = np.reshape(test_gray_image,(len(test_gray_image),SIZE,SIZE,3))
test_color_image = np.reshape(test_color_image, (len(test_color_image),SIZE,SIZE,3))

In [4]:
autoencoder_rgb = keras.models.load_model('E:/TER/model/autoencoder_rgb.h5')

In [5]:
def plot_images(color,grayscale,predicted,n):
    
    M = tf.keras.losses.MeanSquaredError()
    MSE = M(color, predicted)
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.title('Color Image')
    plt.imshow(color)
    plt.subplot(1,3,2)
    plt.title('Grayscale Image ')
    plt.imshow(grayscale)
    plt.subplot(1,3,3)
    plt.title('Predicted Image ')
    plt.imshow(predicted)
    plt.suptitle("MSE: "+str(MSE.numpy()), fontsize = 20)
    plt.imsave('E:/TER/Result_autoencoder_landscape_RGB/output/'+str(n)+".png",predicted)
    plt.savefig('E:/TER/Result_autoencoder_landscape_RGB/plot/'+str(n)+".png")
    plt.close()

predicted = np.zeros((7129,SIZE, SIZE,3))
predicted[:1782] = np.clip(autoencoder_rgb.predict(test_gray_image[:1782].reshape(1782,SIZE, SIZE,3)),0.0,1.0).reshape(1782,SIZE, SIZE,3)
predicted[1782:3564] = (np.clip(autoencoder_rgb.predict(test_gray_image[1782:3564].reshape(1782,SIZE, SIZE,3)),0.0,1.0).reshape(1782,SIZE, SIZE,3))
predicted[3564:5346] = (np.clip(autoencoder_rgb.predict(test_gray_image[3564:5346].reshape(1782,SIZE, SIZE,3)),0.0,1.0).reshape(1782,SIZE, SIZE,3))
predicted[5346:7129] = (np.clip(autoencoder_rgb.predict(test_gray_image[5346:7129].reshape(1783,SIZE, SIZE,3)),0.0,1.0).reshape(1783,SIZE, SIZE,3))
with tqdm(total=7129) as pbar: 
    for i in range(640,650):
        plot_images(test_color_image[i],test_gray_image[i],predicted[i],i)
        pbar.update(1)

56/56 [==============================] - 12s 157ms/step


  0%|          | 10/7129 [00:03<40:02,  2.96it/s] 
